In [ ]:
!pip install torch accelerate bitsandbytes datasets transformers peft trl scipy  deepspeed wandb -q

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_id = "microsoft/Phi-3.5-mini-instruct-onnx"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use float32 for CPU
    device_map="cpu",  # Explicitly set to CPU
    #device_map="cuda",
)

# Start interactive loop
print("Chatbot is ready! Type 'exit' to quit.\n")

# History of the conversation
messages = []

while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    # Append the user's message to the conversation history
    messages.append({"role": "user", "content": user_input})

    # Tokenize input
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"  # Keep tensors on CPU
    )

    # Define terminators
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # Generate outputs
    outputs = model.generate(
        input_ids,
        max_new_tokens=32,  # Adjust for performance
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    # Decode and print the response
    response = outputs[0][input_ids.shape[-1]:]
    bot_response = tokenizer.decode(response, skip_special_tokens=True)
    print(f"Bot: {bot_response.strip()}")

    # Add the bot's response to the conversation history
    messages.append({"role": "assistant", "content": bot_response.strip()})
